In [14]:
import hnswlib
import numpy as np
import pickle
from time import time

In [15]:
dim = 2048
num_elements = 1000

In [16]:
# Generating sample data
data = np.float32(np.random.random((num_elements, dim)))
ids = np.arange(num_elements)

In [17]:
# Declaring index
p = hnswlib.Index(space = 'cosine', dim = dim) # possible options are l2, cosine or ip

In [18]:
# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 200, M = 16)

In [19]:
# Element insertion (can be called several times):
p.add_items(data, ids)

In [20]:
# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

In [53]:
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
ts = []
for _ in range(100):
    start_t = time()
    labels, distances = p.knn_query(data, k = 1)
    ts.append(time() - start_t)
print(np.mean(ts))

0.01730595827102661


In [54]:
ts = []
for _ in range(100):
    start_t = time()
    np.min(np.matmul((data / np.linalg.norm(data, axis=1, keepdims=True)),
                           (data / np.linalg.norm(data, axis=1, keepdims=True)).T), axis=0)
    ts.append(time() - start_t)
print(np.mean(ts))

0.017650837898254394


Verdict: if we do it classwise then we can significantly speed it up. In that case, hnsw may not be significantly faster.